## Machine Learning

### Modelado

- Para nuestro Sistema de Recomendacion utilizamos los filtros basados en contenido (contend-based filtering) como base de la prediccion, es decir, que sugiere películas similares basados en una películas en particular.

- En este sistema, las caracteristicas de las películas (overview, genre, company, actor) se usan para encontrar su similitud con otras películas. Luego se recomiendan 5 películas que tienen más probabilidades de ser similares a la consultada en nuestra API.

- El procesamiento del lenguaje natural (NLP) es un proceso de manipulación o comprensión del texto por cualquier software o máquina.

- Utilizamos La biblioteca NLTK (Natural Language Toolkit), la cual es una suite que contiene bibliotecas para el procesamiento estadístico del lenguaje, y de esta forma hacer que las máquinas entiendan el lenguaje humano y respondan con una respuesta adecuada.

- Tambien utilizamos RAKE, el cual es un algoritmo de extracción de palabras clave en un cuerpo de texto mediante el análisis de la frecuencia de aparición de palabras y su co-ocurrencia con otras palabras en el texto.

- Utilizamos el Stemming, es una técnica de normalización de texto en el procesamiento del lenguaje natural. Esta técnicas es ampliamente utilizadas para el preprocesamiento de texto. Es una técnica en la que un conjunto de palabras en una oración se convierten en una secuencia para acortar su búsqueda. En este método, se normalizan las palabras que tienen el mismo significado pero tienen algunas variaciones según el contexto o la oración.

- Como métrica utilizamos Cosine Similarity, la cual determina cuán similares son los objetos de datos independientemente de su tamaño. En la similitud del coseno, los objetos de datos en un conjunto de datos se tratan como un vector.

- La similitud de coseno es beneficiosa para las aplicaciones que utilizan datos dispersos, como documentos de texto, transacciones en datos de mercado y sistemas de recomendación porque la similitud de coseno ignora las coincidencias 0-0. Contar coincidencias 0-0 en datos escasos inflaría las puntuaciones de similitud.

- Para calcular la similitud del coseno, necesitamos el número de palabras de cada observacion de la columna 'tags'. Para realizar estos calculos utilizamos CountVectorizer de Sklearn para aprender el vocabulario del conjunto de textos y luego transformarlos en un marco de datos que utilizamos para construir el modelo. Este proceso nos da como salida una matriz dispersa 'sparse_matrix', la cual utilizamons en cosine_similarity() y asi obtener el resultado final.

- Adicionalmente creamos una serie con los indices de cada una de de las peliculas.

- En nuestra funcion de recomendacion, pasamos como parametros el titulo de la pelicula que deseamos, y tambien la variable cosine_sim. Se realiza la busqueda del titulo en el archivo de indices, el indice de la misma es buscado en cosine_sim y nos da como resultado una lista de las 5 peliculas similares, dependiendo del score de la pelicula consultada. Si la pelicula no se encuentra en el set de datos, retorna una notificacion.

### Observaciones
- La calidad y cantidad de los datos es de suma importancia. Un buen algoritmo con datos de poca calidad ofrecerá asimismo recomendaciones de baja calidad. Pero unos buenos datos, con el volumen suficiente y organizados eficientemente, nos darán recomendaciones razonablemente buenas aunque el algoritmo no sea óptimo.



- Importamos las librerias necesarias

In [1]:
import pandas as pd
import rake_nltk
from rake_nltk import Rake
import nltk #Natural Language ToolKit
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer #para convertir las caracteristicas combinadas en una matriz
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\58424\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\58424\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
#Importamos el dataset
movies = pd.read_csv('datasets/movies.csv', sep=';', encoding='utf-8')

- Utilizamos la libreria Rake para extraer las palabras claves de cada película, de la columna 'overview'. Creamos la columna 'words' para almacenar las palabras extraídas.

In [13]:
movies['words'] = ""

for index, row in movies.iterrows():
    r = Rake()  #Instancia de Rake
    overview = row['overview']
    r.extract_keywords_from_text(overview)  #Metodo para extraer palabras
    score_words = r.get_word_degrees()    #Diccionario con las palabras claves y sus puntuaciones
    movies.at[index, 'words'] = list(score_words.keys()) #Asignamos las palabras claves a la columna words

In [14]:
movies['genres'] = movies['genre'].str.split(',')
movies['companies'] = movies['company'].str.split(',')
movies['actors'] = movies['actor'].str.split(',')

#Remplazamos los valos " " por "" en cada una de las listas de las columnas desanidadas
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['companies'] = movies['companies'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['actors'] = movies['actors'].apply(lambda x: [i.replace(" ", "") for i in x])

#Creamos una columna que reuna los datos de las columnas que usamos para alimentar el modelo
movies['tags'] = movies['words'] + movies['genres'] + movies['companies'] + movies['actors']

# Se dejan unicamente la columna 'tags' en el modelo puesto que es la unica que se va a ausar para la alimentacion del modelo
movies = movies[['title','tags']]

# unimos todas las cadenas en una sola, con respecto a la columna 'tags'
movies['tags'] = movies['tags'].apply(lambda x:' '.join(x))

In [15]:
# Implementacion de la libreria para normalizar la columna 'tags'
ps = PorterStemmer()

def normalization(text):
    tag = []
    for i in text.split():
        tag.append(ps.stem(i))
    return " ".join(tag)

# aplicamos la funcion sobre la columana 'tags'
movies['tags'] = movies['tags'].apply(normalization)

In [16]:
movies.to_csv('datasets/moviesML.csv', index=True)

In [17]:
count_vectorizer  = CountVectorizer()
sparse_matrix  = count_vectorizer .fit_transform(movies['tags'])
cosine_sim = cosine_similarity(sparse_matrix , sparse_matrix)
print(cosine_sim)

[[1.         0.05484543 0.02299002 ... 0.         0.04767313 0.        ]
 [0.05484543 1.         0.01849317 ... 0.01849317 0.01917412 0.        ]
 [0.02299002 0.01849317 1.         ... 0.         0.09644856 0.03569722]
 ...
 [0.         0.01849317 0.         ... 1.         0.02411214 0.        ]
 [0.04767313 0.01917412 0.09644856 ... 0.02411214 1.         0.01850583]
 [0.         0.         0.03569722 ... 0.         0.01850583 1.        ]]


In [18]:
indices = pd.Series(movies['title'])
indices.to_csv('datasets/indices.csv', index=True)

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

In [9]:
def recomendacion(titulo, cosine_sim = cosine_sim):

    title=titulo.lower()

    #Validar que el titulo ingresado se encuentra en el dataframe
    if titulo not in movies['title'].values:
        return 'Pelicula no encontrada!'
    else:
        list_movies = []
        idx = indices[indices == titulo].index[0]
        score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
        top_5_indices = list(score_series.iloc[1:6].index)
        
        for i in top_5_indices:
            list_movies.append(list(movies['title'])[i])
            
        return list_movies

In [10]:
recomendacion('cars')

['cars 3', 'cars 2', "a bug's life", 'monsters, inc.', 'monsters university']

In [11]:
recomendacion('batman')

['batman & robin',
 'batman forever',
 'batman returns',
 'the lego batman movie',
 'batman: mask of the phantasm']

In [182]:
recomendacion('dracula')

['dracula 2000',
 'van helsing',
 "it's alive",
 "mary shelley's frankenstein",
 'vampire in brooklyn']

In [183]:
recomendacion('jurassic park')

['jurassic world',
 'the lost world: jurassic park',
 'scooby-doo',
 'zathura: a space adventure',
 'rumble fish']

In [184]:
recomendacion('superman')

['superman ii',
 'superman iv: the quest for peace',
 'superman iii',
 'superman returns',
 'max steel']

In [185]:
recomendacion('toy story')

['toy story 2', 'toy story 3', 'free birds', "a bug's life", 'monsters, inc.']

In [186]:
recomendacion('hotel transylvania')

['hotel transylvania 2',
 'grown ups',
 'monster house',
 'grown ups 2',
 'the croods']

In [187]:
recomendacion('star wars')

['the empire strikes back',
 'return of the jedi',
 'star wars: the force awakens',
 'star wars: episode iii - revenge of the sith',
 'star wars: episode i - the phantom menace']

In [188]:
recomendacion('transformers')

['transformers: revenge of the fallen',
 'transformers: dark of the moon',
 'transformers: age of extinction',
 'transformers: the last knight',
 'double dragon']

In [201]:
recomendacion('back to the future')

['back to the future part ii',
 'back to the future part iii',
 'spacecamp',
 'honey, i shrunk the kids',
 'hot tub time machine']